In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from os import listdir
from pickle import dump
from keras.applications.xception import Xception
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.xception import preprocess_input
from keras.models import Model
import string
from tqdm.notebook import tqdm
from pickle import load
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers  import Input,Dense,Flatten,LSTM, GRU, Bidirectional, Embedding,add,Dropout,Conv2D
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [4]:
from keras.models import load_model
models = load_model("/content/modelA_3.h5")

In [15]:
def extract_features(filename):
	# load the model
	model = Xception()
	# re-structure the model
	model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
	# load the photo
	image = load_img(filename, target_size=(299, 299))
	# convert the image pixels to a numpy array
	image = img_to_array(image)
	# reshape data for the model
	image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
	# prepare the image for the VGG model
	image = preprocess_input(image)
	# get features
	feature = model.predict(image, verbose=0)
	return feature

In [16]:
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
             if index == integer:
                     return word
    return None

In [17]:
def generate_desc(model, tokenizer, photo, max_length):
    # seed the generation process
    in_text = 'startseq'
     # iterate over the whole length of the sequence
    for i in range(max_length):
        # integer encode input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        # pad input
        sequence = pad_sequences([sequence], maxlen=max_length)
        # predict next word
        yhat = model.predict([photo,sequence], verbose=0)
        # convert probability to integer
        yhat = np.argmax(yhat)
        # map integer to word
        word = word_for_id(yhat, tokenizer)
        # stop if we cannot map the word
        if word is None:
            break
             # append as input for generating the next word
        in_text += ' ' + word
        # stop if we predict the end of the sequence
        if word == 'endseq':
            break
    return in_text

In [20]:
pic = extract_features("/content/1.jpg") #path of picture 
tokenizer = load(open('/content/tokenizer.pkl', 'rb')) #path to tokenizer file
caption = generate_desc(models, tokenizer, pic, 31)

In [21]:
print(caption)

startseq brown dog is running through the grass endseq
